In [1]:
import argparse
import json
import pytorch_lightning as pl
from argparse import Namespace
from model import DTSModel
from datamodules.csvdatamodule import CsvDataModule
from datamodules.hivedatamodule import HiveDataModule
from datamodules.sqldatamodule import SqlDataModule
from datamodules.s3datamodule import S3DataModule
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from dataclasses import dataclass
import torch

In [2]:
@dataclass
class Config:
    data: dict
    model: dict
    train:dict
        
config = Config(
    data= {
        "data": "csv",
        "data_params": {
            "normalize": 0,
            "dataset": "tst",
            "num_feature": 5,
            "enc_seq_len": 30,
            "dec_seq_len": 5,
            "tgt_seq_len": 5,
            "batch_size": 64,
            "train_path": "data/AAPL_train.csv",
            "val_path": "data/AAPL_val.csv",
            "test_path": "data/AAPL_test.csv"
        }
    },
    model= {
        "model_name": "tst",
        "model_params": {
            "input_size": 5,
            "enc_seq_len": 30,
            "dec_seq_len": 5,
            "out_seq_len": 5,
            "dim_val": 512,
            "n_encoder_layers": 4,
            "n_decoder_layers": 4,
            "n_heads": 8,
            "dropout_encoder": 0.2,
            "dropout_decoder": 0.2,
            "dropout_pos_enc": 0.1,
            "dim_feedforward_encoder": 2048,
            "dim_feedforward_decoder": 2048,
            "num_predicted_features": 1,
            "lr": 1e-4
        },
        "loss_fn_type": "mse",
        "loss_params": {}
    },
    train=  {
        "accelerator": "gpu",
        "devices": 1,
        "strategy": 'ddp',
        "max_epochs": 50
    }
)

In [3]:
pl.seed_everything(42, workers=True)
logger = TensorBoardLogger('logs/', name=config.model['model_name'])

Global seed set to 42


In [4]:
# Create LightningDataModule
data_module = CsvDataModule(config.data['data_params'])


In [5]:
test = data_module.train_data[:,4].reshape(-1, 1)
test.shape

(2959, 1)

In [6]:
next(iter(data_module.train_dataloader()))

[[tensor([[[-0.8288, -0.8302, -0.8245, -0.8481, -0.8370],
           [-0.8289, -0.8319, -0.8285, -0.8523, -0.7107],
           [-0.8353, -0.8350, -0.8327, -0.8552, -0.6602],
           ...,
           [-0.8268, -0.8270, -0.8222, -0.8438, -0.8314],
           [-0.8239, -0.8222, -0.8190, -0.8406, -0.7792],
           [-0.8190, -0.8184, -0.8141, -0.8380, -0.7124]],
  
          [[-0.9246, -0.9273, -0.9237, -0.9369, -0.6629],
           [-0.9268, -0.9283, -0.9251, -0.9376, -0.7026],
           [-0.9255, -0.9258, -0.9236, -0.9356, -0.5890],
           ...,
           [-0.9081, -0.9081, -0.9055, -0.9190, -0.4118],
           [-0.9062, -0.9056, -0.9035, -0.9185, -0.1930],
           [-0.9031, -0.9052, -0.9010, -0.9169, -0.3418]],
  
          [[-0.8521, -0.8508, -0.8480, -0.8690, -0.6821],
           [-0.8474, -0.8502, -0.8454, -0.8688, -0.7630],
           [-0.8472, -0.8492, -0.8432, -0.8678, -0.8290],
           ...,
           [-0.8160, -0.8178, -0.8120, -0.8410, -0.6227],
           [-0.8

In [7]:
# Create LightningModule
device = torch.device('cuda') if config.train['accelerator']=='gpu' else torch.device('cpu')
model = DTSModel(config.model, device)
model.to(device)

DTSModel(
  (model): TSTModel(
    (encoder_input_layer): Linear(in_features=5, out_features=512, bias=True)
    (decoder_input_layer): Linear(in_features=5, out_features=512, bias=True)
    (linear_mapping): Linear(in_features=512, out_features=1, bias=True)
    (positional_encoding_layer): PositionalEncoder(
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
          )
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (dropout): Dropout(p=0.2, inplace=False)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(

In [8]:
# Callback to save the model checkpoint
checkpoint_callback = ModelCheckpoint(
        monitor='val_loss',
        dirpath='./checkpoints/',
        filename='model-{epoch:02d}-{val_loss:.2f}',
        save_top_k=3,
        mode='min',
        save_last=True,
        every_n_epochs=1,  # Save checkpoint every epoch
    )

# Create Trainer
trainer = pl.Trainer(
        accelerator=config.train['accelerator'],
        devices=config.train['devices'],
        #strategy=config.train['strategy'],
        max_epochs=config.train['max_epochs'],
        callbacks=[
            EarlyStopping(monitor='val_loss',patience=5,verbose=False, mode='min'),
            LearningRateMonitor(logging_interval='step'),
            checkpoint_callback
        ],
        logger=logger,
    )

# Train the model
trainer.fit(model, datamodule=data_module)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/hd1/anaconda3/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:613: UserWarning: Checkpoint directory /hd1/dl/deep_time_series_framework/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type     | Params
-------------------------------------
0 | model   | TSTModel | 29.4 M
1 | loss_fn | MSE      | 0     
-------------------------------------
29.4 M    Trainable params
0         Non-trainable params
29.4 M    Total params
117.729   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/hd1/anaconda3/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1609: PossibleUserWarning: The number of training batches (46) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [9]:
preds = trainer.test(datamodule=data_module)

/hd1/anaconda3/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:124: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
Restoring states from the checkpoint path at /hd1/dl/deep_time_series_framework/checkpoints/model-epoch=26-val_loss=0.01.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /hd1/dl/deep_time_series_framework/checkpoints/model-epoch=26-val_loss=0.01.ckpt


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss          0.015542013570666313
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [10]:
preds

[{'test_loss': 0.015542013570666313}]

In [13]:
import numpy as np
test_loader = data_module.val_dataloader()
predictions = []
model.to(device)
for batch_idx, (inputs, labels) in enumerate(test_loader):
    if batch_idx !=0: continue
    
    outputs = model((inputs[0].to(device),inputs[1].to(device)))
    if config.data['data_params']['normalize']:
        outputs = data_module.target_denormalize(outputs.detach().cpu().numpy().squeeze()[:,-1])
        labels = data_module.target_denormalize(labels.detach().cpu().numpy().squeeze()[:,-1])
    else:
        outputs = outputs.detach().cpu().numpy().squeeze()[:,-1]
        labels = labels.detach().cpu().numpy().squeeze()[:,-1]

    predictions.append(list(zip(labels, outputs)))
#predictions = np.concatenate(predictions, axis=0)

In [14]:
predictions

[[(128.3253173828125, 125.42912),
  (124.50135803222656, 138.39554),
  (124.3630142211914, 132.16508),
  (123.8590850830078, 123.34227),
  (119.55094909667967, 123.2675),
  (119.81771850585938, 118.131615),
  (126.2700653076172, 114.279945),
  (123.63182067871094, 127.98467),
  (120.60820770263672, 122.024345),
  (118.70115661621094, 124.391754),
  (119.97582244873048, 117.91181),
  (114.97601318359376, 118.30691),
  (119.64974212646484, 119.624825),
  (118.55294799804688, 123.337),
  (120.5093994140625, 123.46673),
  (119.59046173095705, 113.30608),
  (122.51525115966795, 109.20789),
  (124.0764617919922, 130.36186),
  (123.2761001586914, 121.60758),
  (119.09642028808594, 121.1298),
  (118.56283569335938, 123.4035),
  (121.92239379882812, 118.10034),
  (121.08251190185547, 120.71583),
  (118.66165161132812, 114.06972),
  (119.15569305419922, 115.39009),
  (119.76832580566406, 123.80464),
  (119.9461898803711, 129.58725),
  (118.47390747070312, 116.17007),
  (120.6971435546875, 106.70